## Remove duplicate hits, keep only the best one ( smaller edge ) first to come out



In [88]:
import numpy
from sklearn import preprocessing


def removeDuplicates(df):
    print("## REMOVE DUPLICATES EDGES ##")
    df.sort_values('node1')
    df.drop_duplicates(keep="first", inplace=True)
    print("Lenght of the dataframe after REMOVE DUPLICATES EDGES: " + str(len(df)))
    if len(df) == 0:
        print("ERROR: Lenght of the dataframe = 0 - I can't generate the gephi/cytoscape network")
        exit()
    print('------------------------------')
    return(df)


def removeSelfHit(df):
    print("## REMOVE SELF HITS ##")
    """ this function removes the BLAST self hits """
    to_drop = []
    for i in range(len(df)):
        node1 = df.iloc[i]['node1']
        node2 = df.iloc[i]['node2']
        if str(node1) == str(node2):
            to_drop.append(i)
    df = df.drop(df.index[to_drop])
    print("Lenght of the dataframe after REMOVE SELF HITS: " + str(len(df)))
    if len(df) == 0:
        print("ERROR: Lenght of the dataframe = 0 - I can't generate the gephi/cytoscape network")
        exit()
    print('------------------------------')
    return(df)



In [89]:
import pandas as pd

inputf = "test.tsv"
df = pd.read_csv(inputf, sep='\t', header=None)
df.columns = ["node1","node2","eval"]

print(df)
print("\nLEN DF",len(df))

    node1  node2   eval
0   node1  node2  0.222
1   node1  node2  0.400
2   node1  node2  9.000
3   node2  node1  3.000
4   node2  node1  9.000
5   node2  node1  9.000
6   node2  node1  9.000
7   node2  node1  9.000
8   node1  node2  0.400
9   node1  node2  0.400
10  node1  node2  0.400
11  node1  node2  0.400
12  node1  node1  1.000
13  node2  node2  2.000

LEN DF 14


In [90]:
df = removeSelfHit(df)
print(df)

## REMOVE SELF HITS ##
Lenght of the dataframe after REMOVE SELF HITS: 12
------------------------------
    node1  node2   eval
0   node1  node2  0.222
1   node1  node2  0.400
2   node1  node2  9.000
3   node2  node1  3.000
4   node2  node1  9.000
5   node2  node1  9.000
6   node2  node1  9.000
7   node2  node1  9.000
8   node1  node2  0.400
9   node1  node2  0.400
10  node1  node2  0.400
11  node1  node2  0.400


In [91]:
df = removeDuplicates(df)
print(df)

## REMOVE DUPLICATES EDGES ##
Lenght of the dataframe after REMOVE DUPLICATES EDGES: 5
------------------------------
   node1  node2   eval
0  node1  node2  0.222
1  node1  node2  0.400
2  node1  node2  9.000
3  node2  node1  3.000
4  node2  node1  9.000


In [92]:
def removeBigEval(df):
    print("## KEEP ONLY HIT WITH SMALLER EVAL ##")
    df.sort_values('eval')
    df.drop_duplicates(subset = ['node1', 'node2'], keep="first", inplace=True)
    print("Lenght of the dataframe after KEEP ONLY HIT WITH SMALLER EVAL: " + str(len(df)))
    if len(df) == 0:
        print("ERROR: Lenght of the dataframe = 0 - I can't generate the gephi/cytoscape network")
        exit()
    print('------------------------------')
    return(df)

def evalToForce(df):
    print("Converting Evalues to Forces")
    x = df['eval'].values
    listForces = []
    min = numpy.amin(x)
    max = numpy.amax(x)
    print(f"Min Evalue {min}, Max Evalue {max}")
    for eval in x:
        force = max - (eval - min)
        listForces.append(force)
    df['eval'] = listForces
    return df

def normalize(df):
    ''' CHECKED ---- '''
    print("## NORMALIZING EDGES ##")
    """ Normalisation of the column eval """
    x = df['eval']#.values.astype(float)
    x = x.to_frame()
    #scaler = preprocessing.StandardScaler()
    scaler = preprocessing.MinMaxScaler()
    x_scaled = scaler.fit_transform(x)
    df_normalized = pd.DataFrame(x_scaled)
    dflist = df_normalized.values
    df['eval'] = dflist
    return df

In [93]:
df = normalize(df)
print(df)

## NORMALIZING EDGES ##
   node1  node2      eval
0  node1  node2  0.000000
1  node1  node2  0.020278
2  node1  node2  1.000000
3  node2  node1  0.316473
4  node2  node1  1.000000


In [80]:
df = removeBigEval(df)
print(df)

## KEEP ONLY HIT WITH SMALLER EVAL ##
Lenght of the dataframe after KEEP ONLY HIT WITH SMALLER EVAL: 2
------------------------------
   node1  node2   eval
0  node1  node2  0.222
3  node2  node1  3.000


In [81]:
df = evalToForce(df)
print(df)

Converting Evalues to Forces
Min Evalue 0.222, Max Evalue 3.0
   node1  node2   eval
0  node1  node2  3.000
3  node2  node1  0.222
